#Финальная работа. Часть 1_a. Сбор и преобразование данных для примененияя метода Undersampling.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyspark

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Подключение Spark сессии

In [ ]:
!pip install pyspark --quiet
!pip install -U -q PyDrive --quiet
!apt install openjdk-8-jdk-headless &> /dev/null
!pip install pyspark[sql]

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local[*]")\
        .appName('PySpark_Tutorial')\
        .getOrCreate()


In [ ]:
spark

Загрузим  файл train_data_6.pq

In [ ]:
import pyarrow.parquet as pq
import pyspark.sql.functions as f
df_test = spark.read.parquet("/content/drive/MyDrive/Ml_Junior/Финальная работа/train_data_6.pq")

In [ ]:
df_test.show()

+-------+---+----------------+-------------------+---------+---------+---------------+---------------+----------------------+-----------------------+---------------------+-----------------------+-------------------------+--------------------------+----------+------------+-------------+-------------+-----------+--------------+----------------+-----------------+-----------------+---------------+--------+--------------+-----------------+------------+------------------+---------------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------------------------+-----------------------+---------------------+---------------------+-----------+-----------+
|     id| rn|pre_since_opened|pre_since_confirmed|pre_pterm|pre_fterm|pre_till_pclose|pre_till_

In [ ]:
df_test.printSchema()

root
 |-- id: long (nullable = true)
 |-- rn: long (nullable = true)
 |-- pre_since_opened: long (nullable = true)
 |-- pre_since_confirmed: long (nullable = true)
 |-- pre_pterm: long (nullable = true)
 |-- pre_fterm: long (nullable = true)
 |-- pre_till_pclose: long (nullable = true)
 |-- pre_till_fclose: long (nullable = true)
 |-- pre_loans_credit_limit: long (nullable = true)
 |-- pre_loans_next_pay_summ: long (nullable = true)
 |-- pre_loans_outstanding: long (nullable = true)
 |-- pre_loans_total_overdue: long (nullable = true)
 |-- pre_loans_max_overdue_sum: long (nullable = true)
 |-- pre_loans_credit_cost_rate: long (nullable = true)
 |-- pre_loans5: long (nullable = true)
 |-- pre_loans530: long (nullable = true)
 |-- pre_loans3060: long (nullable = true)
 |-- pre_loans6090: long (nullable = true)
 |-- pre_loans90: long (nullable = true)
 |-- is_zero_loans5: long (nullable = true)
 |-- is_zero_loans530: long (nullable = true)
 |-- is_zero_loans3060: long (nullable = true)
 |

Сгруппируем данные по id клиентов и расчитаем среднее значение по признакам для отложенной выборки

In [ ]:
df_test = df_test.groupby(['id']).mean()
df_test.show()

+-------+---------+-------+---------------------+------------------------+------------------+------------------+--------------------+--------------------+---------------------------+----------------------------+--------------------------+----------------------------+------------------------------+-------------------------------+---------------+------------------+------------------+------------------+----------------+-------------------+---------------------+----------------------+----------------------+--------------------+------------------+-------------------+----------------------+------------------+-----------------------+--------------------------+-------------------+--------------------+-------------------+-------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+----------

In [ ]:
df_test.count()

250000

In [ ]:
df_test = df_test.drop('avg(id)')

In [ ]:
df_test = df_test.to_pandas_on_spark(index_col='id')

/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:5725: FutureWarning: DataFrame.to_pandas_on_spark is deprecated. Use DataFrame.pandas_api instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [ ]:
df_test = df_test.round()

In [ ]:
df_test.columns = df_test.columns.str.replace('avg(', '').str.replace(')', '')

In [ ]:
df_test_new = df_test.to_spark(index_col='id')

In [ ]:
df_test_new.show(5)

+-------+---+----------------+-------------------+---------+---------+---------------+---------------+----------------------+-----------------------+---------------------+-----------------------+-------------------------+--------------------------+----------+------------+-------------+-------------+-----------+--------------+----------------+-----------------+-----------------+---------------+--------+--------------+-----------------+------------+------------------+---------------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------------------------+-----------------------+---------------------+---------------------+-----------+-----------+
|     id| rn|pre_since_opened|pre_since_confirmed|pre_pterm|pre_fterm|pre_till_pclose|pre_till_

Загрузим файл с целевой переменной

In [ ]:
train_target = spark.read.format("csv").option("header", "true").load("/content/drive/MyDrive/Ml_Junior/Финальная работа/train_target.csv")
train_target.show()

+---+----+
| id|flag|
+---+----+
|  0|   0|
|  1|   0|
|  2|   0|
|  3|   0|
|  4|   0|
|  5|   0|
|  6|   0|
|  7|   0|
|  8|   0|
|  9|   0|
| 10|   0|
| 11|   0|
| 12|   0|
| 13|   0|
| 14|   0|
| 15|   0|
| 16|   0|
| 17|   0|
| 18|   0|
| 19|   0|
+---+----+
only showing top 20 rows



Объеденим файл с данными отложенной выборки и файл с целевым признаком

In [ ]:
from pyspark.sql.functions import *

In [ ]:
final_test = df_test_new.join(train_target, 'id', "left")
final_test.show()

+-------+----+----------------+-------------------+---------+---------+---------------+---------------+----------------------+-----------------------+---------------------+-----------------------+-------------------------+--------------------------+----------+------------+-------------+-------------+-----------+--------------+----------------+-----------------+-----------------+---------------+--------+--------------+-----------------+------------+------------------+---------------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------------------------+-----------------------+---------------------+---------------------+-----------+-----------+----+
|     id|  rn|pre_since_opened|pre_since_confirmed|pre_pterm|pre_fterm|pre_till_pclose|pr

Сохраним в формате csv для использования в модели как отложенные данные при премененияя метода Undersampling

In [ ]:
final_test.coalesce(1).write.mode('overwrite').csv("/content/drive/MyDrive/Ml_Junior/Финальная работа/Итоговая работа_все файлы/Spark_для_отбора_признаков/data_test_us.csv", header=True)

Загрузим все остальные файлы train_data 0-4 для формирования данных для обучения и кросс валидации при премененияя метода Undersampling

In [ ]:
parquetFile_0 = spark.read.parquet("/content/drive/MyDrive/Ml_Junior/Финальная работа/train_data_0.pq")
parquetFile_1 = spark.read.parquet("/content/drive/MyDrive/Ml_Junior/Финальная работа/train_data_1.pq")
parquetFile_2 = spark.read.parquet("/content/drive/MyDrive/Ml_Junior/Финальная работа/train_data_2.pq")
parquetFile_3 = spark.read.parquet("/content/drive/MyDrive/Ml_Junior/Финальная работа/train_data_3.pq")
parquetFile_4 = spark.read.parquet("/content/drive/MyDrive/Ml_Junior/Финальная работа/train_data_4.pq")

Объеденим все в один файл

In [ ]:
df_train = parquetFile_0.union(parquetFile_1).union(parquetFile_2).union(parquetFile_3).union(parquetFile_4)
df_train.show()

+---+---+----------------+-------------------+---------+---------+---------------+---------------+----------------------+-----------------------+---------------------+-----------------------+-------------------------+--------------------------+----------+------------+-------------+-------------+-----------+--------------+----------------+-----------------+-----------------+---------------+--------+--------------+-----------------+------------+------------------+---------------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------------------------+-----------------------+---------------------+---------------------+-----------+-----------+
| id| rn|pre_since_opened|pre_since_confirmed|pre_pterm|pre_fterm|pre_till_pclose|pre_till_fclose|p

Сгруппируем данные по id клиентов и расчитаем среднее значение по признакам

In [ ]:
df_train = df_train.groupby(['id']).mean()
df_train.show()

+----+-------+-------+---------------------+------------------------+------------------+------------------+--------------------+--------------------+---------------------------+----------------------------+--------------------------+----------------------------+------------------------------+-------------------------------+---------------+-----------------+------------------+------------------+----------------+-------------------+---------------------+----------------------+----------------------+--------------------+------------------+-------------------+----------------------+------------------+-----------------------+--------------------------+-------------------+-------------------+-------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------

In [ ]:
df_train.count()

1250000

In [ ]:
df_train = df_train.drop('avg(id)')

In [ ]:
df_train = df_train.to_pandas_on_spark(index_col='id')

/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:5725: FutureWarning: DataFrame.to_pandas_on_spark is deprecated. Use DataFrame.pandas_api instead.
  warnings.warn(


In [ ]:
df_train = df_train.round()

In [ ]:
df_train.columns = df_train.columns.str.replace('avg(', '').str.replace(')', '')

In [ ]:
df_train_new = df_train.to_spark(index_col='id')

In [ ]:
df_train_new.show(5)

+----+---+----------------+-------------------+---------+---------+---------------+---------------+----------------------+-----------------------+---------------------+-----------------------+-------------------------+--------------------------+----------+------------+-------------+-------------+-----------+--------------+----------------+-----------------+-----------------+---------------+--------+--------------+-----------------+------------+------------------+---------------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------------------------+-----------------------+---------------------+---------------------+-----------+-----------+
|  id| rn|pre_since_opened|pre_since_confirmed|pre_pterm|pre_fterm|pre_till_pclose|pre_till_fclose

Объеденим файл train с данными и файл с целевым признаком

In [ ]:
final_train = df_train_new.join(train_target, 'id', "left")
final_train.show()

+----+----+----------------+-------------------+---------+---------+---------------+---------------+----------------------+-----------------------+---------------------+-----------------------+-------------------------+--------------------------+----------+------------+-------------+-------------+-----------+--------------+----------------+-----------------+-----------------+---------------+--------+--------------+-----------------+------------+------------------+---------------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------------------------+-----------------------+---------------------+---------------------+-----------+-----------+----+
|  id|  rn|pre_since_opened|pre_since_confirmed|pre_pterm|pre_fterm|pre_till_pclose|pre_till

Запишем итоговый файл в формат CSV. Данный файл я буду использовать при премененияя метода Undersampling.

In [ ]:
final_train.coalesce(1).write.mode('overwrite').csv("/content/drive/MyDrive/Ml_Junior/Финальная работа/Итоговая работа_все файлы/Spark_для_отбора_признаков/data_train_us.csv", header=True)

|